In [3]:
import numpy as np
import pandas as pd
import numba

## Read in/Format Data

In [4]:
clair = pd.read_csv("CSV/clair-de-lune.csv", skiprows = 192, header = None).dropna(axis = 0)
dvorak = pd.read_csv("CSV/Dvorak9Largo.csv", skiprows = 98, header = None).dropna(axis = 0)
jupiter = pd.read_csv("CSV/Jupiter.csv", skiprows = 39, header = None).dropna(axis = 0)
pachelbel = pd.read_csv("CSV/pachelbel.csv", skiprows = 27, header = None).dropna(axis = 0)

In [5]:
#Select Notes and velocity columns#
Cnotes = clair.ix[:,4]
Cvelocity = clair.ix[:,5]

#Find possible unique notes and velocities
CpossibleNotes = np.unique(Cnotes)
CpossibleVelocities =  np.unique(Cvelocity)


#Select Notes and velocity columns#
Dnotes = dvorak.ix[:,4]
Dvelocity = dvorak.ix[:,5]

#Find possible unique notes and velocities
DpossibleNotes = np.unique(Dnotes)
DpossibleVelocities =  np.unique(Dvelocity)

#Select Notes and velocity columns#
Jnotes = jupiter.ix[:,4]
Jvelocity = jupiter.ix[:,5]

#Find possible unique notes and velocities
JpossibleNotes = np.unique(Jnotes)
JpossibleVelocities =  np.unique(Jvelocity)

#Select Notes and velocity columns#
Pnotes = pachelbel.ix[:,4]
Pvelocity = pachelbel.ix[:,5]

#Find possible unique notes and velocities
PpossibleNotes = np.unique(Pnotes)
PpossibleVelocities =  np.unique(Pvelocity)

In [6]:
Ck1 = len(CpossibleNotes)
Ck2 = len(CpossibleVelocities)

Dk1 = len(DpossibleNotes)
Dk2 = len(DpossibleVelocities)

Jk1 = len(JpossibleNotes)
Jk2 = len(JpossibleVelocities)

Pk1 = len(PpossibleNotes)
Pk2 = len(PpossibleVelocities)


In [1]:
def encode(x, code):
    output = np.zeros(len(x))
    for i in range(0, len(x)):
        output[i] = int(np.where(code == x[i])[0]) 
    return output

In [7]:
CxNotes = encode(Cnotes, CpossibleNotes)
CxVel = encode(Cvelocity, CpossibleVelocities)

DxNotes = encode(Dnotes, DpossibleNotes)
DxVel = encode(Dvelocity, DpossibleVelocities)

JxNotes = encode(Jnotes, JpossibleNotes)
JxVel = encode(Jvelocity, JpossibleVelocities)

PxNotes = encode(Pnotes, PpossibleNotes)
PxVel = encode(Pvelocity, PpossibleVelocities)

## 1st Order HMM - Estimate pi, phi, T

In [39]:
#@numba.njit()
#Function using the log-sum-exp trick#
def logSumExp(a):
    b = np.max(a)
    return(b + np.log(np.sum(np.exp(a-b))))

#####################
##Forward Algorithm##
#####################

#Function to run forward algorithm, arguments are n = # obs, m = # states for z,#
#k = # states for x, pi = initial distribution(m vector), 
#Tmat = transition matrix (mxm), phi = emission distribution (m x k matrix)#
#x is the observed data#
#takes log of pi, Tmat and phi
@numba.jit()
def forwardAlg(n, m, k, pi, Tmat, phi, x):
    g = np.zeros((n,m))
    for i in range(0,m):
        g[0,i] = (pi[i]) + (phi[i, x[0]])
    
    for j in range(1, n):
        for l in range(0, m):
            g[j,l] = logSumExp(g[j-1, :]+(Tmat[:,l])+(phi[l,x[j]-1]))
    return(g)

def pForward(g, x):
    pXf = logSumExp(g[len(x)-1,:])
    return(pXf)

In [40]:
@numba.jit()
def backwardAlg(n, m, k, pi, Tmat, phi, x):
    r = np.zeros((n,m))
    for j in range(n-2, -1, -1):
        for l in range(0, m):
            r[j, l] = logSumExp(r[j+1,: ] + Tmat[l,:] + phi[:, x[j+1]])
    
    return(r)
@numba.jit()
#Function to return p(x_1:n) from matrix from backward algorithm
def pBackward(r, pi, phi, x):
    pXb = logSumExp(r[0,: ]+ pi +phi[:,x[0]])
    return(pXb)

In [41]:
@numba.jit()
def BaumWelch(n, m, k, x, tol):
    #randomly initialize pi, phi and T#
    vals = np.random.rand(m)
    pi = np.log(vals/np.sum(vals))
    Tmat = np.zeros(shape = (m, m))
    phi = np.zeros(shape = (m, k))
    for i in range(0, m):
        vals1 = np.random.rand(m)
        Tmat[i, ] = np.log(vals1/np.sum(vals1))
        vals2 = np.random.rand(k)
        phi[i, ] = np.log(vals2/np.sum(vals2))
    
    iterations = 0
    convergence = 0
    pOld = 1E10
    
    #Initialize matricies for gamma and beta values#
    gamma = np.zeros(shape = (n, m))
    beta = np.zeros(shape = (n,m,m))
    
    #Stop iterations when log(p(x_1:n)) differs by tol between iterations#
    while convergence == 0:
        #Perform forward and backward algorithms# 
        g = forwardAlg(n, m, k, pi, Tmat, phi, x)
        h = backwardAlg(n, m, k, pi, Tmat, phi, x)
        pNew = pForward(g, x)
        
        ##E-Step##
    
        #Calculate gamma and beta#
        for t in range(0, n):
            gamma[t,] = g[t,] + h[t,] - pNew
        for t in range(0, n):
            for i in range(0, m):
                for j in range(0, m):
                    if t == 1:
                        beta[t,i,j] = 1
                    else:
                        beta[t,i,j] = Tmat[i,j] + phi[j, x[t]] + g[t-1, i] + h[t, j] - pNew
        ##M-Step##
    
        #Update pi, phi and Tmat#
        pi = gamma[0,] - logSumExp(gamma[0,])
        for i in range(0, m):
            for j in range(0, m):
                Tmat[i,j] = logSumExp(beta[range(1, n), i, j]) - logSumExp(beta[range(1,n), i, ])
        for i in range(0,m):
            for w in range(0, k):
                indicies = np.where(x == w)
                phi[i,w] = logSumExp(gamma[indicies, i]) - logSumExp(gamma[:,i])
        
        criteria = abs(pOld - pNew)
        if criteria < tol:
            convergence = 1
        else:
            convergence = 0
            pOld = pNew
            iterations +=1
        return (iterations, pNew, np.exp(pi), np.exp(phi), np.exp(Tmat))
        

In [53]:
np.random.seed(77)
Jit1, Jp1, Jpi1, Jphi1, JTmat1 = BaumWelch(Jn1, 10, Jk1, JxNotes, 0.0001)
Jit2, Jp2, Jpi2, Jphi2, JTmat2 = BaumWelch(Jn2, 10, Jk2, JxVel, 0.0001)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


In [70]:
np.random.seed(77)
Cit1, Cp1, Cpi1, Cphi1, CTmat1 = BaumWelch(Cn1, 50, Ck1, CxNotes, 0.0001)
Cit2, Cp2, Cpi2, Cphi2, CTmat2 = BaumWelch(Cn2, 50, Ck2, CxVel, 0.0001)

Dit1, Dp1, Dpi1, Dphi1, DTmat1 = BaumWelch(Dn1, 50, Dk1, DxNotes, 0.0001)
Dit2, Dp2, Dpi2, Dphi2, DTmat2 = BaumWelch(Dn2, 50, Dk2, DxVel, 0.0001)

Jit1, Jp1, Jpi1, Jphi1, JTmat1 = BaumWelch(Jn1, 50, Jk1, JxNotes, 0.0001)
Jit2, Jp2, Jpi2, Jphi2, JTmat2 = BaumWelch(Jn2, 50, Jk2, JxVel, 0.0001)

Pit1, Pp1, Ppi1, Pphi1, PTmat1 = BaumWelch(Pn1, 50, Pk1, PxNotes, 0.0001)
Pit2, Pp2, Ppi2, Pphi2, PTmat2 = BaumWelch(Pn2, 50, Pk2, PxVel, 0.0001)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: DeprecationWarning: using a non-integer number instead of an integer will result in

## 2nd Order HMM - Estimate T2

In [34]:
#@numba.njit()
#Function using the log-sum-exp trick#
def logSumExp(a):
    b = np.max(a)
    return(b + np.log(np.sum(np.exp(a-b))))

#####################
##Forward Algorithm##
#####################

#Function to run forward algorithm, arguments are n = # obs, m = # states for z,#
#k = # states for x, pi = initial distribution(m vector), 
#Tmat = transition matrix (mxm), phi = emission distribution (m x k matrix)#
#x is the observed data#
#takes log of pi, Tmat and phi
@numba.jit()
def forwardAlg2(n, m, k, pi, Tmat, T2mat, phi, x):
    g = np.zeros(m)
    alpha = np.zeros((n, m, m))
    
    g = pi + phi[:, x[0]]
    
    for t in range(1,n):
        for j in range(0,m):
            for l in range(0,m):
                if t ==1:
                    alpha[1,j,l] = g[j] + Tmat[j,l] + phi[l, x[1]]
                else:
                    alpha[t,j,l] = logSumExp(alpha[t-1,:,j] + T2mat[:,j,l] + phi[l, x[t]])
    return alpha

def pForward2(m,g, x):
    pXf = logSumExp(g[len(x)-1,:,m-1])
    return(pXf)

In [35]:
m = 10
Cn1 = len(CxNotes)
Cn2 = len(CxVel)

Dn1 = len(DxNotes)
Dn2 = len(DxVel)

Jn1 = len(JxNotes)
Jn2 = len(JxVel)

Pn1 = len(PxNotes)
Pn2 = len(PxVel)

k1 = len(JpossibleNotes)


pi1 = np.full(m, 1/m)
phi1 = np.full((m,k1), 1/k1)
Tmat1 = np.full((m,m), 1/m)
T2mat1 = np.full((m,m,m), 1/m)
G1 = forwardAlg2(Jn1, m, k1, np.log(pi1), np.log(Tmat1), np.log(T2mat1), np.log(phi1), JxNotes)
p1 = pForward2(m, G1, JxNotes)
p1

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:21: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


-3845.2483708539939

In [20]:
k1
#G1[len(JxNotes)-1,:,k1]

27

In [36]:
@numba.jit()
def backwardAlg2(n, m, k, pi, Tmat, T2mat, phi, x):
    beta = np.zeros((n,m,m))
    for t in range(n-2, -1, -1):
        for j in range(0, m):
            for l in range(0,m):
                beta[t,j, l] = logSumExp(beta[t+1,j,: ] + T2mat[j,l,:] + phi[:, x[j+1]])
    
    return(beta)
@numba.jit()
#Function to return p(x_1:n) from matrix from backward algorithm
def pBackward2(m,r, pi, phi, x):
    pXb = logSumExp(r[0,:,m-1]+ pi +phi[:,x[0]])
    return(pXb)

In [37]:
G2 = backwardAlg2(Jn1, m, k1, np.log(pi1), np.log(Tmat1), np.log(T2mat1), np.log(phi1), JxNotes)
p2 = pBackward2(m, G2, np.log(pi1), np.log(phi1), JxNotes)
p2

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  if __name__ == '__main__':
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


-3842.945785761

In [58]:
G2.shape

(1166, 10, 10)

In [61]:
@numba.jit()
def BaumWelch2(n, m, k, x, pi, Tmat, phi, tol):
    #randomly initialize T2mat#
    T2mat = np.zeros(shape = (m,m,m))
    for i in range(0, m):
        for j in range(0,m):
            vals = np.random.rand(m)
            T2mat[i,j, ] = np.log(vals/np.sum(vals))
    iterations = 0
    convergence = 0
    pOld = 1E10
    
    #Initialize matricies for gamma and beta values#
    gamma = np.zeros(shape = (n, m))
    beta = np.zeros(shape = (n,m,m))
    #Stop iterations when log(p(x_1:n)) differs by tol between iterations#
    while convergence == 0:
        
        #Perform forward and backward algorithms# 
        alpha = forwardAlg2(n, m, k, pi, Tmat, T2mat, phi, x)
        beta = backwardAlg2(n, m, k, pi, Tmat, T2mat, phi, x)
        pNew = pForward2(m,alpha, x)
        ##M-Step##
        eta = np.zeros((n,m,m,m))
        #Update pi, phi and Tmat#

        for t in range(1, n-1):
            for i in range(0, m):
                for j in range(0, m):
                    for l in range(0,m):
                        eta[t,i,j,l] = alpha[t,i,j] + T2mat[i,j,l] + phi[l, x[t+1]] + beta[t+1, j, l] - pNew
        
        for i in range(0, m):
            for j in range(0, m):
                for l in range(0,m):
                        T2mat[i,j,l] = logSumExp(eta[range(1,n),i,j,l]) - logSumExp(eta[range(1,n),i,j,])
        print(iterations)
        criteria = abs(pOld - pNew)
        if criteria < tol:
            convergence = 1
        else:
            convergence = 0
            pOld = pNew
            iterations +=1
        return (iterations, np.exp(T2mat))
        

In [62]:
np.random.seed(77)
Jit1, JT2mat1 = BaumWelch2(Jn1, 10, Jk1, JxNotes, Jpi1, JTmat1, Jphi1, 0.01)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


0


In [64]:
np.random.seed(77)
Jit1, JT2mat1 = BaumWelch2(Jn1, 30, Jk1, JxNotes, Jpi1, JTmat1, Jphi1, 0.001)
Jit2, JT2mat2 = BaumWelch2(Jn2, 30, Jk2, JxVel, Jpi2, JTmat2, Jphi2, 0.001)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


0


/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


0


In [71]:
np.random.seed(77)
Cit1, CT2mat1 = BaumWelch2(Cn1, 50, Ck1, CxNotes, Cpi1, CTmat1, Cphi1, 0.001)
Cit2, CT2mat2 = BaumWelch2(Cn2, 50, Ck2, CxVel, Cpi2, CTmat2, Cphi2, 0.001)

Dit1, DT2mat1 = BaumWelch2(Dn1, 50, Dk1, DxNotes, Dpi1, DTmat1, Dphi1, 0.001)
Dit2, DT2mat2 = BaumWelch2(Dn2, 50, Dk2, DxVel, Dpi2, DTmat2, Dphi2, 0.001)

Jit1, JT2mat1 = BaumWelch2(Jn1, 50, Jk1, JxNotes, Jpi1, JTmat1, Jphi1, 0.001)
Jit2, JT2mat2 = BaumWelch2(Jn2, 50, Jk2, JxVel, Jpi2, JTmat2, Jphi2, 0.001)

Pit1, PT2mat1 = BaumWelch2(Pn1, 50, Pk1, PxNotes, Ppi1, PTmat1, Pphi1, 0.001)
Pit2, PT2mat2 = BaumWelch2(Pn2, 50, Pk2, PxVel, Ppi2, PTmat2, Pphi2, 0.001)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  from ipykernel import kernelapp as app


0


/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


0


/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


...

0


/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0


/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


KeyboardInterrupt: 

In [79]:
np.random.seed(77)

Jit1, JT2mat1 = BaumWelch2(Jn1, 50, Jk1, JxNotes, Jpi1, JTmat1, Jphi1, 0.001)
Jit2, JT2mat2 = BaumWelch2(Jn2, 50, Jk2, JxVel, Jpi2, JTmat2, Jphi2, 0.001)

Pit1, PT2mat1 = BaumWelch2(Pn1, 50, Pk1, PxNotes, Ppi1, PTmat1, Pphi1, 0.001)
Pit2, PT2mat2 = BaumWelch2(Pn2, 50, Pk2, PxVel, Ppi2, PTmat2, Pphi2, 0.001)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  app.launch_new_instance()


0


/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0


/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:6: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0


/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:7: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


0


In [65]:
def decode(x, code):
    output = np.zeros(len(x))
    for i in range(0, len(x)):
        output[i] = code[x[i]]
    return output

In [66]:
def hmm(n, pi, phi, Tmat, T2mat, code):
    m = Tmat.shape[0]
    k = phi.shape[1]
    zstates = range(0, m)
    xstates = range(0, k)
    z = np.zeros(n)
    x = np.zeros(n)
    z[0] = np.random.choice(zstates, size = 1,  p = pi)
    z[1] = np.random.choice(zstates, size = 1,  p = Tmat[z[0], :])
    for j in range(2, n):
        z[j] = np.random.choice(zstates, size = 1,  p = T2mat[z[j-2],z[j-1], :])
    for i in range(0, n):
        x[i] = np.random.choice(xstates, size =1, p = phi[z[i], :])
    output = decode(x, code)
    return output


In [34]:
zstates = range(0, m)
Jpi1

0.011695355479488755

In [76]:
DT2mat2.shape

(50, 50, 50)

In [77]:
CnewNotes = hmm(Cn1, Cpi1, Cphi1, CTmat1, CT2mat1, CpossibleNotes)
CnewVelocities = hmm(Cn2, Cpi2, Cphi2, CTmat2, CT2mat2, CpossibleVelocities)

DnewNotes = hmm(Dn1, Dpi1, Dphi1, DTmat1, DT2mat1, DpossibleNotes)
DnewVelocities = hmm(Dn2, Dpi2, Dphi2, DTmat2, DT2mat2, DpossibleVelocities)

# JnewNotes = hmm(Jn1, Jpi1, Jphi1, JTmat1, JT2mat1, JpossibleNotes)
# JnewVelocities = hmm(Jn2, Jpi2, Jphi2, JTmat2, JT2mat2,JpossibleVelocities)

# PnewNotes = hmm(Pn1, Ppi1, Pphi1, PTmat1, PpossibleNotes)
# PnewVelocities = hmm(Pn2, Ppi2, Pphi2, PTmat2, PpossibleVelocities)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [81]:
JnewNotes = hmm(Jn1, Jpi1, Jphi1, JTmat1, JT2mat1, JpossibleNotes)
JnewVelocities = hmm(Jn2, Jpi2, Jphi2, JTmat2, JT2mat2,JpossibleVelocities)

PnewNotes = hmm(Pn1, Ppi1, Pphi1, PTmat1, PT2mat1, PpossibleNotes)
PnewVelocities = hmm(Pn2, Ppi2, Pphi2, PTmat2, PT2mat2, PpossibleVelocities)

/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:9: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:11: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:13: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
/Users/annayanchenko/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


In [78]:
Coutput = pd.DataFrame(CnewNotes)
Coutput["vel"] = CnewVelocities
Coutput.to_csv("clair-de-luneRemix2.csv")

Doutput = pd.DataFrame(DnewNotes)
Doutput["vel"] = DnewVelocities
Doutput.to_csv("Dvorak9Remix2.csv")

# Joutput = pd.DataFrame(JnewNotes)
# Joutput["vel"] = JnewVelocities
# Joutput.to_csv("jupiterRemix2.csv")

# Poutput = pd.DataFrame(PnewNotes)
# Poutput["vel"] = PnewVelocities
# Poutput.to_csv("pachelbelRemix.csv")

In [82]:
Joutput = pd.DataFrame(JnewNotes)
Joutput["vel"] = JnewVelocities
Joutput.to_csv("jupiterRemix2.csv")

Poutput = pd.DataFrame(PnewNotes)
Poutput["vel"] = PnewVelocities
Poutput.to_csv("pachelbelRemix2.csv")